In [4]:
def create_features(years, lags_ID, lags_DA):

    import pandas as pd

    # Intraday price (ID)

    ID = pd.read_pickle(f"../epex/ID.pkl")

    ID = ID.astype('float')

    ID = ID.loc[ID.index.year.isin(years)]

    # Day-ahead price (DA)

    # DA = client.query_day_ahead_prices(country_code, start = start, end = end)

    DA = pd.read_pickle(f"../epex/DA.pkl")

    DA = DA.astype('float')

    DA = DA.loc[DA.index.year.isin(years)]

    features = pd.merge(ID, DA, left_index = True, right_index = True, how = 'outer')

    # Load (LOAD)

    # LOAD = client.query_load(country_code, start = start, end = end)
    
    LOAD = pd.read_csv('./LOAD_NL.csv', index_col = 0)

    LOAD.index.names = ['Date']

    LOAD = LOAD.rename(columns = {'Forecast':'Load forecast'})

    LOAD.index = pd.to_datetime(LOAD.index)

    LOAD = LOAD.resample('1H').mean()

    features = pd.merge(features, LOAD, left_index = True, right_index = True, how = 'outer')

    # Lags

    for l in lags_ID:
        features[f'ID3 ({l})'] = features['ID3'].shift(l)
    
    for l in lags_DA:
        features[f'MCP ({l})'] = features['MCP'].shift(l)

    # Generation

    features.index = features.index.tz_localize('UTC')

    GEN = pd.read_csv('./GEN.csv', index_col = 0)

    features = pd.merge(features, GEN, left_index = True, right_index = True, how = 'outer')

    # Environment

    ENV = pd.read_csv('./ENV.csv', index_col = 0)

    features = pd.merge(features, ENV, left_index = True, right_index = True, how = 'outer')

    # Daily dummies

    for d in range(0, 7):
        features[f'DOW {d}'] = 0
        features.loc[features.index.weekday == d, f'DOW {d}'] = 1

    for h in range(0, 24):
        features[f'HOD {h}'] = 0
        features.loc[features.index.hour == h, f'HOD {h}'] = 1

    features = features.dropna()

    return ID, DA, features

In [31]:
import pandas as pd

from entsoe import EntsoeRawClient
from entsoe import EntsoePandasClient

client = EntsoePandasClient(api_key = '9ee80e1b-31ad-41b3-addc-1533c2b8227f')

years = [2018]

start = pd.Timestamp(f'{years[0]}.01.01')
end = pd.Timestamp(f'{years[-1]}.12.31')
country_code = 'NL'  # Netherlands

GEN = client.query_generation_forecast(country_code, start = start, end = end)

GEN = GEN.to_frame()

GEN = GEN.rename(columns = {0:'Gen. forecast'})

GEN.index = GEN.index.tz_convert('UTC')

GEN = GEN.resample('1H').mean()

GEN.to_csv('./GEN.csv')

ENV = client.query_wind_and_solar_forecast(country_code, start = start, end = end, psr_type = None)

ENV = ENV.rename(columns = {'Solar':'Gen. (s) forecast', 'Wind Offshore':'Gen. (w_off) forecast', 'Wind Onshore':'Gen. (w_on) forecast'})

ENV.index = ENV.index.tz_convert('UTC')

ENV = ENV.resample('1H').mean()

ENV.to_csv('./ENV.csv')